In [1]:
from gensim.models import Word2Vec, FastText
import glob
from note_reader import get_notes

In [2]:
file_list = glob.glob("data/txt/*.txt")

In [3]:
docs = get_notes(file_list, as_pitch_classes=True)

In [4]:
w2v_model = Word2Vec(sentences=docs, vector_size=50, epochs=10)

In [5]:
w2v_model.wv.most_similar(positive=["G"])

[('F#4', 0.4300253689289093),
 ('Eb4', 0.2511135935783386),
 ('Eb1', 0.19390258193016052),
 ('Bb5', 0.15681758522987366),
 ('Eb5', 0.15323102474212646),
 ('Ab4', 0.08846789598464966),
 ('A', 0.08022525161504745),
 ('E#4', 0.057010289281606674),
 ('C#4', 0.012033085338771343),
 ('Bb4', 0.011051268316805363)]

In [6]:
ft_model = FastText(sentences=docs, vector_size=50, epochs=10)

In [8]:
ft_model.wv.most_similar("G")

[('F#4', 0.41548749804496765),
 ('Eb4', 0.19267885386943817),
 ('Bb5', 0.16282042860984802),
 ('Ab4', 0.1499929130077362),
 ('Eb1', 0.14080627262592316),
 ('E#4', 0.12632432579994202),
 ('Eb5', 0.06039716675877571),
 ('A', 0.03681078553199768),
 ('Bb4', 0.01627650111913681),
 ('Cb4', -0.00244335038587451)]